# Gold Layer - Star Schema Builder

## Overview
This notebook builds analytics-ready star schemas (Gold layer) from Silver tables.

**Star Schemas Created:**
- Conformed Dimensions (DimDate, DimCustomer, DimProduct, DimEmployee)
- Fact Tables optimized for Direct Lake

**Prerequisites:**
- Silver tables created (run 02_transform_to_silver.ipynb first)

**Output:**
- Gold star schema tables ready for Power BI Direct Lake

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from datetime import datetime

print(f"Gold Star Schema Builder Started: {datetime.now()}")

## 🔍 Diagnostic - Prerequisites Check

In [ ]:
# ============================================================================
# DIAGNOSTIC - Check prerequisites for Gold layer creation
# ============================================================================

print("="*80)
print("DIAGNOSTIC - Checking prerequisites for Gold layer")
print("="*80)

# 1. List all tables in catalog
all_tables_list = spark.catalog.listTables()
print(f"\n1️⃣ Total tables in catalog: {len(all_tables_list)}")

if len(all_tables_list) == 0:
    print("\n❌ ERROR: No tables found in lakehouse!")
    print("   ACTION REQUIRED:")
    print("   1. Upload CSV files to Files/bronze/")
    print("   2. Run notebook 01_ingest_to_bronze.ipynb")
    print("   3. Run notebook 02_transform_to_silver.ipynb")
else:
    # 2. Check Bronze dimensions
    bronze_dims = [t.name for t in all_tables_list 
                   if t.name.lower().startswith("dim") 
                   and not t.name.lower().startswith("silver_")
                   and not t.name.lower().startswith("gold_")]
    
    print(f"\n2️⃣ Bronze dimensions found: {len(bronze_dims)}")
    if bronze_dims:
        for dim in sorted(bronze_dims):
            df = spark.table(dim)
            print(f"   ✅ {dim:25s} ({df.count():,} rows)")
    else:
        print("   ⚠️  No Bronze dimensions found")
    
    # 3. Check Silver dimensions
    silver_dims = [t.name for t in all_tables_list 
                   if t.name.lower().startswith("silver_dim")]
    
    print(f"\n3️⃣ Silver dimensions found: {len(silver_dims)}")
    if silver_dims:
        for dim in sorted(silver_dims):
            df = spark.table(dim)
            print(f"   ✅ {dim:35s} ({df.count():,} rows)")
    else:
        print("   ⚠️  No Silver dimensions found")
        print("   ACTION: Run notebook 02_transform_to_silver.ipynb first")
    
    # 4. Check Silver facts
    silver_facts = [t.name for t in all_tables_list 
                    if t.name.lower().startswith("silver_fact")]
    
    print(f"\n4️⃣ Silver facts found: {len(silver_facts)}")
    if silver_facts:
        for i, fact in enumerate(sorted(silver_facts), 1):
            df = spark.table(fact)
            print(f"   {i:2d}. {fact:35s} ({df.count():,} rows)")
            if i >= 10:
                remaining = len(silver_facts) - 10
                if remaining > 0:
                    print(f"   ... and {remaining} more")
                break
    
    # 5. Check existing Gold tables
    gold_dims = [t.name for t in all_tables_list 
                 if t.name.lower().startswith("gold_dim")]
    gold_facts = [t.name for t in all_tables_list 
                  if t.name.lower().startswith("gold_fact")]
    
    print(f"\n5️⃣ Existing Gold tables: {len(gold_dims) + len(gold_facts)}")
    if gold_dims or gold_facts:
        print("   ⚠️  Gold tables already exist (will be overwritten):")
        for dim in sorted(gold_dims):
            print(f"     {dim}")
        for fact in sorted(gold_facts[:5]):
            print(f"     {fact}")
        if len(gold_facts) > 5:
            print(f"     ... and {len(gold_facts) - 5} more facts")
    else:
        print("   ℹ️  No Gold tables yet (this is expected on first run)")
    
    # 6. Show what will be created
    total_dims = len(bronze_dims) + len(silver_dims)
    total_facts = len(silver_facts)
    
    print(f"\n6️⃣ Tables to be created:")
    print(f"   📊 {total_dims} Gold dimensions (from {len(bronze_dims)} Bronze + {len(silver_dims)} Silver)")
    print(f"   📈 {total_facts} Gold fact tables")
    
    # 7. Final status
    print("\n" + "="*80)
    if len(bronze_dims) > 0 or len(silver_dims) > 0:
        print("✅ READY - Prerequisites met. You can proceed with Gold layer creation.")
    else:
        print("❌ NOT READY - Missing dimension tables.")
        print("   Run notebooks 01 and 02 first to create Bronze and Silver tables.")
    print("="*80)

## Configuration

In [ ]:
# Configuration - Dynamically discover Silver tables and Bronze dimensions
all_tables = spark.catalog.listTables()

print(f"Total tables in catalog: {len(all_tables)}")

if len(all_tables) == 0:
    print("\n⚠️  WARNING: No tables found in lakehouse!")
    print("Please run notebook 02_transform_to_silver.ipynb first.")
    DIMENSION_MAPPINGS = {}
    FACT_MAPPINGS = {}
else:
        
    # Get Silver dimensions
    silver_dims = [t.name for t in all_tables if t.name.startswith("silver_dim")]
    
    # Get Silver facts
    silver_facts = [t.name for t in all_tables if t.name.startswith("silver_fact")]
    
    # Build dimension mappings (Bronze Dim* → Gold Dim*, Silver_Dim* → Gold Dim*)
    DIMENSION_MAPPINGS = {}
    
    # Add Bronze dimensions (DimDate, DimGeography) with Gold_ prefix
    for bronze_dim in bronze_dims:
        gold_name = f"Gold_{bronze_dim}"  # Add Gold_ prefix
        DIMENSION_MAPPINGS[bronze_dim] = gold_name
    
    # Add Silver dimensions (replace silver_ with Gold_)
    for silver_dim in silver_dims:
        gold_name = silver_dim.replace("silver_", "Gold_")
        DIMENSION_MAPPINGS[silver_dim] = gold_name
    
    # Build fact mappings (replace silver_ with Gold_)
    FACT_MAPPINGS = {}
    for silver_fact in silver_facts:
        gold_name = silver_fact.replace("silver_", "Gold_")
        FACT_MAPPINGS[silver_fact] = gold_name
    
    print(f"\n✅ Discovered {len(DIMENSION_MAPPINGS)} dimension tables to build")
    print(f"✅ Discovered {len(FACT_MAPPINGS)} fact tables to build")
    
    print(f"\nDimensions ({len(DIMENSION_MAPPINGS)}):")
    for source, target in sorted(DIMENSION_MAPPINGS.items()):
        print(f"  {source:35s} → {target}")
    
    print(f"\nFacts ({len(FACT_MAPPINGS)}):")
    for i, (source, target) in enumerate(sorted(FACT_MAPPINGS.items()), 1):
        print(f"  {i:2d}. {source:35s} → {target}")
        if i >= 15:
            remaining = len(FACT_MAPPINGS) - 15
            if remaining > 0:
                print(f"  ... and {remaining} more")
            break

## Build Conformed Dimensions (Gold)

In [ ]:
print("\n" + "="*80)
print("STEP 1: Building Dimension Tables (Gold)")
print("="*80)

dimension_results = {}

for source_table, target_table in DIMENSION_MAPPINGS.items():
    try:
        print(f"\nBuilding {target_table}...")
        
        # Read source table (Bronze or Silver)
        df = spark.table(source_table)
        
        # Remove metadata columns if present
        metadata_cols = ["_ingestion_timestamp", "_source_file", "row_num"]
        business_cols = [c for c in df.columns if c not in metadata_cols]
        df_clean = df.select(*business_cols)
        
        row_count = df_clean.count()
        
        # Write to Gold layer
        df_clean.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(target_table)
        
        print(f"✅ {target_table} created: {row_count:,} rows")
        dimension_results[target_table] = True
        
    except Exception as e:
        print(f"❌ Error creating {target_table}: {str(e)}")
        dimension_results[target_table] = False

success_count = len([v for v in dimension_results.values() if v])
print(f"\n✅ Dimension tables created: {success_count}/{len(DIMENSION_MAPPINGS)}")

## Build Fact Tables (Gold)

In [ ]:
print("\n" + "="*80)
print("STEP 2: Building Fact Tables (Gold)")
print("="*80)

for source_table, target_table in FACT_MAPPINGS.items():
    try:
        print(f"\nBuilding {target_table}...")
        
        # Read Silver table
        df = spark.table(source_table)
        
        # Select only business columns
        metadata_cols = ["_ingestion_timestamp", "_source_file", "row_num"]
        business_cols = [c for c in df.columns if c not in metadata_cols]
        df_clean = df.select(*business_cols)
        
        # Add partition column for performance (date-based)
        if "order_date_id" in df_clean.columns:
            df_clean = df_clean.withColumn("year_month", 
                                           substring(col("order_date_id").cast("string"), 1, 6))
        elif "create_date_id" in df_clean.columns:
            df_clean = df_clean.withColumn("year_month",
                                           substring(col("create_date_id").cast("string"), 1, 6))
        
        row_count = df_clean.count()
        
        # Write to Gold layer with partitioning (if applicable)
        if "year_month" in df_clean.columns:
            df_clean.write.format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .partitionBy("year_month") \
                .saveAsTable(target_table)
        else:
            df_clean.write.format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(target_table)
        
        print(f"✅ {target_table} created: {row_count:,} rows")
        
    except Exception as e:
        print(f"⚠️  Skipping {target_table}: {str(e)}")

print("\n✅ Fact tables created")

## Optimize Delta Tables for Direct Lake

In [ ]:
print("\n" + "="*80)
print("STEP 3: Optimizing Delta Tables for Direct Lake")
print("="*80)

# Get all Gold tables (Dim* and Fact*) - case insensitive
gold_tables = [t.name for t in spark.catalog.listTables() 
               if t.name.lower().startswith("gold_dim") or t.name.lower().startswith("gold_fact")]

for table_name in gold_tables:
    try:
        print(f"\nOptimizing {table_name}...")
        
        # Run OPTIMIZE command to compact small files
        spark.sql(f"OPTIMIZE {table_name}")
        
        # Run VACUUM to clean up old files (keep 7 days)
        # Note: In production, adjust retention period as needed
        spark.sql(f"VACUUM {table_name} RETAIN 168 HOURS")
        
        print(f"✅ {table_name} optimized")
        
    except Exception as e:
        print(f"⚠️  Could not optimize {table_name}: {str(e)}")

print("\n✅ Delta table optimization complete")

## Verify Star Schema Relationships

In [ ]:
print("\n" + "="*80)
print("STEP 4: Verifying Star Schema Relationships")
print("="*80)

# Get all Gold fact and dimension tables - case insensitive
gold_facts = [t.name for t in spark.catalog.listTables() 
              if t.name.lower().startswith("gold_fact")]
gold_dims = [t.name for t in spark.catalog.listTables() 
             if t.name.lower().startswith("gold_dim") or t.name.lower().startswith("dim")]

print(f"\n📊 Found {len(gold_facts)} fact tables and {len(gold_dims)} dimension tables")

# Track overall validation results
total_checks = 0
passed_checks = 0
failed_checks = 0

# For each fact table, verify all foreign key relationships
for fact_table_name in sorted(gold_facts):
    try:
        fact_df = spark.table(fact_table_name)
        fact_columns = fact_df.columns
        
        print(f"\n🔍 Checking {fact_table_name}...")
        
        # Find all potential foreign key columns (ending with _id or _key)
        fk_columns = [col for col in fact_columns 
                     if col.endswith('_id') or col.endswith('_key')]
        
        if not fk_columns:
            print(f"   ℹ️  No foreign key columns found")
            continue
        
        # For each foreign key column, try to find matching dimension table
        for fk_col in fk_columns:
            # Try to infer dimension table name
            # Examples: customer_id → gold_dimcustomer, product_id → gold_dimproduct
            # date_id → dimdate, order_date_id → dimdate
            
            possible_dim_names = []
            
            # Extract base name from FK column
            if fk_col.endswith('_date_id'):
                possible_dim_names = ['dimdate', 'gold_dimdate']
            elif fk_col.endswith('_id'):
                base_name = fk_col.replace('_id', '')
                possible_dim_names = [
                    f'gold_dim{base_name}',
                    f'dim{base_name}',
                    f'gold_dim{base_name.replace("_", "")}',
                    f'dim{base_name.replace("_", "")}'
                ]
            elif fk_col.endswith('_key'):
                base_name = fk_col.replace('_key', '')
                possible_dim_names = [
                    f'gold_dim{base_name}',
                    f'dim{base_name}',
                    f'gold_dim{base_name.replace("_", "")}',
                    f'dim{base_name.replace("_", "")}'
                ]
            
            # Find matching dimension table
            matching_dim = None
            for dim_name in possible_dim_names:
                if dim_name in [d.lower() for d in gold_dims]:
                    matching_dim = [d for d in gold_dims if d.lower() == dim_name][0]
                    break
            
            if not matching_dim:
                # Check if the FK might reference a dimension with different naming
                for dim_table in gold_dims:
                    dim_df = spark.table(dim_table)
                    if fk_col in dim_df.columns:
                        matching_dim = dim_table
                        break
            
            if matching_dim:
                total_checks += 1
                try:
                    # Get dimension table
                    dim_df = spark.table(matching_dim)
                    
                    # Find the primary key column in dimension
                    # Try common patterns: same name as FK, or without prefix
                    pk_candidates = [fk_col]
                    if '_' in fk_col:
                        # For order_date_id, try date_id
                        parts = fk_col.split('_')
                        if len(parts) > 2:
                            pk_candidates.append('_'.join(parts[-2:]))
                    
                    pk_col = None
                    for candidate in pk_candidates:
                        if candidate in dim_df.columns:
                            pk_col = candidate
                            break
                    
                    if not pk_col:
                        print(f"   ⚠️  {fk_col} → {matching_dim}: Cannot find PK column")
                        failed_checks += 1
                        continue
                    
                    # Check for orphaned records
                    orphaned = fact_df.select(fk_col).distinct() \
                        .join(dim_df.select(pk_col), 
                              fact_df[fk_col] == dim_df[pk_col], 
                              "left_anti")
                    
                    orphan_count = orphaned.count()
                    
                    if orphan_count == 0:
                        print(f"   ✅ {fk_col} → {matching_dim}.{pk_col}: All keys valid")
                        passed_checks += 1
                    else:
                        print(f"   ⚠️  {fk_col} → {matching_dim}.{pk_col}: {orphan_count} orphaned keys")
                        failed_checks += 1
                        if orphan_count <= 5:
                            print(f"       Orphaned values:")
                            orphaned.show(orphan_count, truncate=False)
                        else:
                            print(f"       Sample orphaned values:")
                            orphaned.show(5, truncate=False)
                
                except Exception as e:
                    print(f"   ❌ Error checking {fk_col} → {matching_dim}: {str(e)}")
                    failed_checks += 1
            else:
                # FK column doesn't match any dimension table
                print(f"   ℹ️  {fk_col}: No matching dimension table found")
    
    except Exception as e:
        print(f"❌ Error processing {fact_table_name}: {str(e)}")

# Print summary
print("\n" + "="*80)
print("REFERENTIAL INTEGRITY VALIDATION SUMMARY")
print("="*80)
print(f"Total relationship checks: {total_checks}")
print(f"✅ Passed: {passed_checks}")
print(f"⚠️  Failed: {failed_checks}")

if failed_checks == 0 and total_checks > 0:
    print("\n🎉 All referential integrity checks passed!")
elif total_checks == 0:
    print("\n⚠️  No relationships could be validated")
else:
    print(f"\n⚠️  {failed_checks} relationship(s) have orphaned records")
    print("   Consider cleaning data or updating dimension tables")

print("="*80)

## Generate Star Schema Summary

In [ ]:
print("\n" + "="*80)
print("GOLD LAYER - STAR SCHEMA SUMMARY")
print("="*80)

# Get all Gold tables dynamically
all_gold_tables = spark.catalog.listTables()

# Separate dimensions and facts
gold_dims = [t for t in all_gold_tables 
             if t.name.startswith("gold_dim") or 
                (t.name.startswith("gold_dim") and not t.name.startswith("gold_"))]
gold_facts = [t for t in all_gold_tables if t.name.startswith("gold_fact")]

# Calculate total statistics
total_dim_rows = 0
total_fact_rows = 0

print("\n📊 DIMENSION TABLES")
print("-" * 80)

if len(gold_dims) == 0:
    print("  ⚠️  No dimension tables found")
else:
    print(f"{'Table Name':<30} | {'Rows':>12} | {'Columns':>8} | {'Size Info':<20}")
    print("-" * 80)
    
    for table in sorted(gold_dims, key=lambda x: x.name):
        try:
            df = spark.table(table.name)
            row_count = df.count()
            col_count = len(df.columns)
            total_dim_rows += row_count
            
            # Get some key column info
            key_cols = [c for c in df.columns if c.endswith('_id') or c.endswith('_key')]
            key_info = f"{len(key_cols)} key col(s)"
            
            print(f"  {table.name:<28} | {row_count:>12,} | {col_count:>8} | {key_info:<20}")
        except Exception as e:
            print(f"  {table.name:<28} | {'ERROR':>12} | {'-':>8} | {str(e)[:20]:<20}")
    
    print("-" * 80)
    print(f"  {'TOTAL DIMENSIONS':<28} | {total_dim_rows:>12,} | {'':<8} | {len(gold_dims)} table(s)")

print("\n📈 FACT TABLES")
print("-" * 80)

if len(gold_facts) == 0:
    print("  ⚠️  No fact tables found")
else:
    print(f"{'Table Name':<30} | {'Rows':>12} | {'Columns':>8} | {'Partitions':<20}")
    print("-" * 80)
    
    for table in sorted(gold_facts, key=lambda x: x.name):
        try:
            df = spark.table(table.name)
            row_count = df.count()
            col_count = len(df.columns)
            total_fact_rows += row_count
            
            # Check for partitioning
            partition_info = "Not partitioned"
            if "year_month" in df.columns:
                partition_count = df.select("year_month").distinct().count()
                partition_info = f"{partition_count} partition(s)"
            
            print(f"  {table.name:<28} | {row_count:>12,} | {col_count:>8} | {partition_info:<20}")
        except Exception as e:
            print(f"  {table.name:<28} | {'ERROR':>12} | {'-':>8} | {str(e)[:20]:<20}")
    
    print("-" * 80)
    print(f"  {'TOTAL FACTS':<28} | {total_fact_rows:>12,} | {'':<8} | {len(gold_facts)} table(s)")

# Overall summary
print("\n" + "="*80)
print("OVERALL STATISTICS")
print("="*80)
print(f"  Total Dimension Tables: {len(gold_dims)}")
print(f"  Total Fact Tables:      {len(gold_facts)}")
print(f"  Total Dimension Rows:   {total_dim_rows:,}")
print(f"  Total Fact Rows:        {total_fact_rows:,}")
print(f"  Total Gold Tables:      {len(gold_dims) + len(gold_facts)}")

if len(gold_dims) > 0 and len(gold_facts) > 0:
    avg_cardinality = total_fact_rows / total_dim_rows if total_dim_rows > 0 else 0
    print(f"  Avg Fact:Dim Ratio:     {avg_cardinality:.2f}:1")

print("\n" + "="*80)
print(f"Completion Time: {datetime.now()}")

# Final status message
if len(gold_dims) > 0 and len(gold_facts) > 0:
    print("\n✅ Gold star schema is ready for Power BI Direct Lake!")
    print("\nNext Steps:")
    print("  1. Create Power BI semantic model with Direct Lake connection")
    print("  2. Add relationships in Power BI model view")
    print("  3. Import DAX measures from fabric/powerbi/dax-measures.md")
    print("  4. Configure Fabric Data Agent")
elif len(gold_facts) > 0 and len(gold_dims) == 0:
    print("\n⚠️  WARNING: Fact tables created but no dimension tables found!")
    print("   Consider running the dimension creation notebook first.")
elif len(gold_dims) > 0 and len(gold_facts) == 0:
    print("\n⚠️  WARNING: Dimension tables created but no fact tables found!")
    print("   Consider running the Silver layer notebook to create fact tables.")
else:
    print("\n⚠️  WARNING: No Gold tables were created!")
    print("   Please check the previous steps for errors.")

print("="*80)


GOLD LAYER - STAR SCHEMA SUMMARY


NameError: name 'spark' is not defined